In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import progressbar
from scipy import linalg as la
from scipy.sparse import identity
from scipy.sparse import rand
from scipy.sparse import diags
from scipy.sparse import triu

def gen_model(dim):
    ## dim is dimension value
    dim_o=dim
    A1 = -rand(dim,dim,density=0.75).toarray()/5
    A2 = triu(A1, k=1).toarray()/(10)
    A = diags(np.random.normal(-0.5,0,dim),0).toarray()/50 + A2 - A2.T
    ## we denote R1^{1/2},R2^{1/2} as R1,R2 repectively for convenience
    ## Non-Identity covariance matrix
    R1=(identity(dim).toarray() + np.tri(dim,dim,1) - np.tri(dim,dim,-2))/2
    R2=(identity(dim_o).toarray() + np.tri(dim_o,dim_o,1) - np.tri(dim_o,dim_o,-2))/2

    H=rand(dim_o,dim,density=0.75).toarray()/20
    m0=np.zeros(dim)+6
    C0=identity(dim).toarray()
    ## Collection of input 
    collection_input = [dim,dim_o,A,R1,R2,H,m0,C0]
    return collection_input




def gen_data(T,l,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**l)
    I=identity(dim).toarray()
    tau=2**(-l)
    L=la.expm(A*tau)
    ## We are going to need W to be symmetric! 
    W=(R1@R1)@(la.inv(A+A.T)@(L@(L.T)-I))
    C=tau*H
    V=(R2@R2)*tau

    v=np.zeros((J+1,dim,1))
    z=np.zeros((J+1,dim_o,1))
    v[0]=np.random.multivariate_normal(m0,C0,(1)).T
    z[0]=np.zeros((dim_o,1))


    for j in range(J):
        ## truth
        v[j+1] = L@v[j] + np.random.multivariate_normal(np.zeros(dim),W,(1)).T
        ## observation
        z[j+1] = z[j] + C@v[j+1] + np.random.multivariate_normal(np.zeros(dim_o),V,(1)).T
        
    return([z,v])

def cut(T,lmax,l,v):
    ind = np.arange(T*2**l+1)
    rtau = 2**(lmax-l)
    w = v[ind*rtau]
    return(w)

def EnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**l)
    I=identity(dim).toarray()
    I_o=identity(dim_o).toarray()
    dt=2**(-l)
    
    m=np.zeros((J+1,dim,1))
    c=np.zeros((J+1,dim,dim))
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    for j in range(J):
        dw = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
        dv = np.random.multivariate_normal(np.zeros(dim_o),dt*I_o,N).T
        ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
        ## x-m[j]:dim*N c[j]:dim*dim
        
        step1=(((x-m[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        step3=step2@( (z[j+1]-z[j]) - (H@x*dt + R2@dv) )
        step4=(x-m[j])@step3 /(N-1)
        
        x = x + A@x*dt + R1@dw + step4
        m[j+1] = (np.mean(x, axis=1)).reshape(dim,1)

    return([m,c])

def CEnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**(l-1))
    I=identity(dim).toarray()
    I1=identity(dim_o).toarray()
    dt=2**(-l)
    dt1=2**(-l+1)
    
    m=np.zeros((J*2+1,dim,1))
    m1=np.zeros((J+1,dim,1))
    c=np.zeros((J*2+1,dim,dim))
    c1=np.zeros((J+1,dim,dim))
    z1=cut(T,lmax,l-1,z)
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    x1 = x
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    m1[0]=m[0]
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    c1[0]=c[0]
    
    dw=np.zeros((2,dim,N))
    dv=np.zeros((2,dim_o,N))
    for j in range(J):
        for s in range(2):
            dw[s] = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
            dv[s] = np.random.multivariate_normal(np.zeros(dim_o),dt*I1,N).T
            ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
            ## x-m[j]:dim*N c[j]:dim*dim

            step1=(((x-m[2*j+s]).T)@(H.T))
            step2=step1@(la.inv(R2)@la.inv(R2))
            step3=step2@( (z[2*j+s+1]-z[2*j+s]) - (H@x*dt + R2@dv[s]) )
            step4=(x-m[2*j+s])@step3 /(N-1)

            x = x + A@x*dt + R1@dw[s] + step4
            m[2*j+s+1] = (np.mean(x, axis=1)).reshape(dim,1)
        
        step1=(((x1-m1[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        step3=step2@( (z1[j+1]-z1[j]) - (H@x1*dt1 + R2@(dv[0]+dv[1])) )
        step4=(x1-m1[j])@step3 /(N-1)
        
        x1 = x1 + A@x1*dt1 + R1@(dw[0]+dw[1]) + step4
        m1[j+1] = (np.mean(x1, axis=1)).reshape(dim,1)
        
    return([m,m1])
  
def coef(x, y): 
    # number of observations/points 
    n = np.size(x) 
  
    # mean of x and y vector 
    m_x, m_y = np.mean(x), np.mean(y) 
  
    # calculating cross-deviation and deviation about x 
    SS_xy = np.sum(y*x) - n*m_y*m_x 
    SS_xx = np.sum(x*x) - n*m_x*m_x 
  
    # calculating regression coefficients 
    b_1 = SS_xy / SS_xx 
    b_0 = m_y - b_1*m_x 
  
    return(b_0, b_1) 

##### For 50 dimensional model

In [ ]:
def gen_model(dim):
    ## dim is dimension value
    dim_o=dim
    A1 = -rand(dim,dim,density=0.75).toarray()/50
    A2 = triu(A1, k=1).toarray()/(100)
    A = diags(np.random.normal(-0.5,0,dim),0).toarray()/500 + (A2 - A2.T)
    ## we denote R1^{1/2},R2^{1/2} as R1,R2 repectively for convenience
    ## Non-Identity covariance matrix
    R1=(identity(dim).toarray() + np.tri(dim,dim,1)/5 - np.tri(dim,dim,-2)/5)/20
    R2=(identity(dim_o).toarray() + np.tri(dim_o,dim_o,1)/5 - np.tri(dim_o,dim_o,-2)/5)/20

    H=rand(dim_o,dim,density=0.75).toarray()/200
    m0=np.zeros(dim)+6
    C0=identity(dim).toarray()
    ## Collection of input 
    collection_input = [dim,dim_o,A,R1,R2,H,m0,C0]
    return collection_input

### Function to check the convergence rate of $\mathbb{E}[(\eta_{t}^{l,N}(\varphi)-\eta_{t}^{l}(\varphi))^2] = \mathcal{O}(\frac{1}{N})$

### We inpute the random.seed value to differentiate between parallel nodes

In [2]:
def fit_var_enkbf(seed_val):
    np.random.seed(seed_val)
    dim = 10
    l=4
    lmax=9
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    num_seq = np.zeros(6)
    for i in range(6):
        num_seq[i] = 10*2**i
        
    Rep=100
    evar=np.zeros(6)
    for numi in range(6):
        est=np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                mean_mat = EnKBF(T,l,lmax,z,int(num_seq[numi]),collection_input)[0]
                est[rep] = np.mean(mean_mat[-1,:])
        evar[numi] = np.var(est)
    
    x = np.log10(num_seq)
    y = np.log10(evar)
    b = coef(x,y)
    return b

### When time T=2, the run takes less than 1 min

### More importantly, different rand seed values give different results

In [3]:
import time
import datetime
stime = time.time()
time.sleep(3)
etime = time.time()
time_len = str(datetime.timedelta(seconds=etime-stime))
print("Time cost is:",time_len)

Time cost is: 0:00:03.003172


In [4]:
T = 2

stime = time.time()

output = fit_var_enkbf(2)
print("output is", output)

etime = time.time()
time_len = str(datetime.timedelta(seconds=etime-stime))
print("Time cost is:",time_len)

output is (0.059519309784551, -1.005931372009187)
Time cost is: 0:00:15.386217


In [4]:
T = 4

stime = time.time()

output = fit_var_enkbf(2)
print("output is", output)

etime = time.time()
time_len = str(datetime.timedelta(seconds=etime-stime))
print("Time cost is:",time_len)

output is (0.5264612299126088, -1.0445494928705368)
Time cost is: 0:00:30.482516


In [29]:
fit_var_enkbf(3)

(0.5231338777469539, -1.086309629416056)

### Multiprocessing implementation of it

### Some rand seed values inevitably provides exploding values, extra warning operations are added to skip the warning info

### We need to filter our the nan values at the end

In [5]:
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

T = 100
nprocs = mp.cpu_count()
print("%d worker available" %nprocs)

# let's create a pool of workers
pool = mp.Pool(processes=nprocs)

64 worker available


### In less than 1 min, I am able to obtain 4 sets of rate results through parallel computation

### For some random seed, we obtain an exploding values, which we can filter out afterwards

In [6]:
stime = time.time()

result = pool.map(fit_var_enkbf,np.arange(2,10))
print("Results collection:")
print(result)
pool.close()

etime = time.time()
time_len = str(datetime.timedelta(seconds=etime-stime))
print("Time cost is:",time_len)

Results collection:
[(1.206895061586577, -0.9040068563179959), (1.0310696625266855, -0.9354824363150225), (1.092138865733789, -0.9614493432091288), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (0.7202896661449978, -0.8374373575391668)]
Time cost is: 0:40:24.516960


In [26]:
53.8/8

6.725

### Convert result to numpy array and filter out NaN values

In [8]:
x = np.asarray(result)
len_x = int(x[~np.isnan(x)].shape[0]/2)
result_array = x[~np.isnan(x)].reshape((len_x,2))

result_array

array([[ 1.20689506, -0.90400686],
       [ 1.03106966, -0.93548244],
       [ 1.09213887, -0.96144934],
       [ 0.72028967, -0.83743736]])